In [1]:
from sklearn.model_selection import KFold, train_test_split
import torch.distributions.distribution as D
import data.medain_filtering_class as mf
from torch.utils.data import DataLoader
from torch.autograd import Variable
from new_RBM import new_RBM as RBM
import matplotlib.pyplot as plt
import data.read_samples as rs
import torch.optim as optim
import numpy as np
import datetime
import torch

print(datetime.datetime.now(), "model.py code start")

BATCH_SIZE = 10
EPOCH = 100
LEARNING_RATE = 0.2
ANNEALING_RATE = 0.999
VISIBLE_UNITS = [180, 200, 250]
HIDDEN_UNITS = [80, 100, 120]
K_FOLD = 1

[INFO] Pre-processing for make clean
2022-05-12 22:21:48.572910 model.py code start


In [2]:
print("[MODL] Model main code is starting....")

print("[INFO] Read train data, cross-vaildation data and test data from median filtering code")
db1_sig, db1_label, db2_sig, db2_label, db3_sig, db3_label = rs.return_list()

[MODL] Model main code is starting....
[INFO] Read train data, cross-vaildation data and test data from median filtering code
[INFO] Read records file from  ./data/db1/
[RSLT]			 Export records ...
		 ['100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '111', '112', '113', '114', '115', '116', '117', '118', '119', '121', '122', '123', '124']
[INFO]./rdsamp commending start
[IWIP]		rdsamp Converting 100
[IWIP]		rdsamp Converting 101
[IWIP]		rdsamp Converting 102
[IWIP]		rdsamp Converting 103
[IWIP]		rdsamp Converting 104
[IWIP]		rdsamp Converting 105
[IWIP]		rdsamp Converting 106
[IWIP]		rdsamp Converting 107
[IWIP]		rdsamp Converting 108
[IWIP]		rdsamp Converting 109
[IWIP]		rdsamp Converting 111
[IWIP]		rdsamp Converting 112
[IWIP]		rdsamp Converting 113
[IWIP]		rdsamp Converting 114
[IWIP]		rdsamp Converting 115
[IWIP]		rdsamp Converting 116
[IWIP]		rdsamp Converting 117
[IWIP]		rdsamp Converting 118
[IWIP]		rdsamp Converting 119
[IWIP]		rdsamp Converting 121
[IWIP

In [3]:
# X_train, X_test, y_train, y_test = train_test_split(
#     (db1_sig + db2_sig), 
#     (db1_label + db2_label),
#     test_size=0.33,
#     shuffle=True
# )

train_dataloader = DataLoader(db1_sig + db2_sig,
                              batch_size=BATCH_SIZE,
                              num_workers=0, 
                              collate_fn=lambda x: x,
                              shuffle=True)
for i, j in enumerate(train_dataloader):
    print((i), len(j[0]), len(j[1]), len(j[2]))

# test_dataloader = DataLoader(X_test,
#                              batch_size=BATCH_SIZE,
#                              num_workers=0, 
#                              collate_fn=lambda x: x,
#                              shuffle=True)

0 1300000 1300000 1300000
1 1300000 1300000 1300000
2 1300000 1300000 1300000
3 1300000 1300000 1300000
4 1300000 1300000 1300000


In [4]:
rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE)

first_train_op = optim.SGD(rbm_first.parameters(), 0.1)
second_train_op = optim.SGD(rbm_second.parameters(), 0.1)
third_train_op = optim.SGD(rbm_third.parameters(), 0.1)

gb_first_train_op = optim.SGD(rbm_first.parameters(), 0.1)
gb_second_train_op = optim.SGD(rbm_second.parameters(), 0.1)
gb_third_train_op = optim.SGD(rbm_third.parameters(), 0.1)

output_from_first = list()
output_from_second = list()
output_from_third = list()

In [5]:
'''Train Part'''

loss_ = []
for epoch in range(EPOCH):
    '''First bbrbm'''
    for i, (data) in enumerate(train_dataloader):
        print(i, len(data))
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)
        
        sample_data = torch.bernoulli(data)
        sample_data = torch.flatten(sample_data.clone())

        # tensor binary
        vog_first, v1, mt = rbm_first(sample_data)
        
        loss_first = rbm_first.free_energy(vog_first) - rbm_first.free_energy(v1)
        loss_.append(loss_first.data)
        
        first_train_op.zero_grad()
        loss_first.backward()
        first_train_op.step()
    
    output_from_first.append(v1.tolist())
    print("1ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, np.mean(loss_), mt))


0 [array([-0.30615146, -0.41909453, -0.52818075, ..., -1.09967682,
       -1.18483203, -1.27376821]), array([-0.0453678 , -0.03803879, -0.03108191, ..., -0.1199187 ,
       -0.06109926, -0.00026691]), array([ 4.55783847e-02, -4.69140562e-02, -1.35335608e-01, ...,
       -3.78736461e-02, -1.91565441e-02,  1.34895152e-04]), array([-0.09567127, -0.07987886, -0.06488157, ..., -0.00481061,
       -0.00197898,  0.00092321]), array([-0.42988348, -0.44171328, -0.45306181, ..., -0.14964782,
       -0.07737323, -0.00241335]), array([-0.01989794,  0.00142091,  0.02174554, ..., -0.06352973,
       -0.03169495,  0.00121785]), array([-0.13942461, -0.16769816, -0.19464199, ..., -0.01631253,
       -0.00809489,  0.00044248]), array([-0.44599713, -0.36336111, -0.28448549, ...,  0.06662756,
        0.03532796,  0.00171623]), array([-0.09390527, -0.08143038, -0.0696448 , ..., -0.03108182,
       -0.015121  ,  0.00110692]), array([-2.04888850e+00, -1.95987777e+00, -1.87399689e+00, ...,
       -1.14950271e

C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_8624/2993726462.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  torch.tensor(data, dtype=torch.float32)
c:\Users\HILAB_Labtop_02\anaconda3\lib\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


1 [array([-0.15029988, -0.11791669, -0.08648237, ...,  0.02906648,
        0.01470476, -0.0002708 ]), array([-3.45031590e-01, -3.23924506e-01, -3.03754515e-01, ...,
       -8.10659940e-02, -4.13510299e-02, -2.31444459e-04]), array([-0.99653239, -0.93409305, -0.87454898, ..., -0.7976964 ,
       -1.02733887, -1.26683338]), array([-0.14521376, -0.13613053, -0.12744043, ..., -0.25443756,
       -0.11872065,  0.02279599]), array([-0.00042045,  0.01453708,  0.02879427, ..., -0.02954279,
       -0.01555392, -0.00103926]), array([-0.03516251, -0.02767874, -0.02052272, ..., -0.86240796,
       -1.06109412, -1.26738461]), array([-2.45825949e-01, -1.91818094e-01, -1.40314968e-01, ...,
       -1.35721174e-02, -6.74898170e-03,  2.23347612e-04]), array([-0.50012461, -0.43795964, -0.37858267, ..., -0.01281872,
       -0.00584569,  0.00110559]), array([-2.60618982e-01, -2.54660112e-01, -2.48916606e-01, ...,
       -4.39259576e-02, -2.23485650e-02, -3.77202594e-05]), array([-0.92568429, -0.8734416 , -

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x10400000 and 13000000x1)